In [1]:
import warnings
warnings.filterwarnings("ignore")
!pip install rich

In [2]:
import numpy as np
from tqdm import trange
import pandas as pd
import numpy as np
from rich.console import Console
from IPython.display import clear_output 

#Read Dataset

In [3]:
cons = Console()

df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',
                 names=['user_id', 'item_id', 'rating', 'timestamp'])

r = df.pivot(index='user_id', columns='item_id', values='rating').values
r

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 5., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  5., nan, ..., nan, nan, nan]])

# Regression Model

## Loss Function

S = {(i, j): $r_{i,j}$ is observed}

We will make predictions according to the biases of each user and item. Hence the estimated value:

$$\hat{r}_{i,j} = \beta^{user}_i + \beta^{item}_j$$




$$Loss Function: {\dfrac{1}{2}} \sum_{i,j \in S }  (r_{i,j} - \beta^{user}_i - \beta^{item}_j)^2 = {\dfrac{1}{2}} \sum_{i,j \in S }  (r_{i,j} - \hat{r}_{i,j})^2$$

In [4]:
def loss(r: np.ndarray, userbias: np.ndarray, itembias: np.ndarray, i: int, j: int):
  return ((r[i, j] - userbias[i] - itembias[j]) ** 2) / 2

##Gradient

When we take the derivative of the Loss Function with respect to the $\beta^{user}$ to calculate the gradient, it is equal to the derivative with respect to the $\beta^{item}$: <br><br>


$$ \frac{\partial L}{\partial \beta^{user}} = \frac{\partial L}{\partial \beta^{item}} =  -\sum_{i,j \in S }  (r_{i,j} - \hat{r}_{i,j}) = -\sum_{i,j \in S }  (r_{i,j} - \beta^{user}_i - \beta^{item}_j)$$


In [5]:
def gradient(r: np.ndarray, userbias: np.ndarray, itembias: np.ndarray, i: int, j: int):
  return -(r[i, j] - userbias[i] - itembias[j])

##Regression Model Training

In [6]:
def fitLinearRegression(r: np.ndarray, userbias: np.ndarray, itembias: np.ndarray, irow: np.ndarray, jcol: np.ndarray, learning_rate: float = 0.001, epoch: int = 200) -> np.ndarray:
    for t in trange(epoch):
        userbias_prev = userbias.copy()
        itembias_prev = itembias.copy()
        lossArr = []
        for i, j in zip(irow, jcol):
          if np.isnan(r[i, j]):
            continue
          lossArr.append(loss(r, userbias, itembias, i, j))
          # Stochastic Gradient Descent
          gradientUI = gradient(r, userbias, itembias, i, j)
          userbias[i] = userbias[i] - learning_rate * gradientUI
          itembias[j] = itembias[j] - learning_rate * gradientUI
        
        if (t+1) % 20 == 0:
          print(f"iteration {t+1}, Total Loss: {np.sum(lossArr)}")
        
        if np.linalg.norm(userbias - userbias_prev) < 0.0001 or np.linalg.norm(itembias - itembias_prev) < 0.0001:
          print(f"I do early stopping at iteration {t+1}")
          break
    
    return userbias, itembias

##Train-Test Split

While splitting the Train-Test, the values reserved for the test set are changed to nan.

In [7]:
def train_test_split(r, ratio):
  #index of non-empty elements
  irow, jcol = np.where(~np.isnan(r))
  cons.print(f"{len(irow)} entries available")

  idx = np.random.choice(np.arange(len(irow)), int(len(irow) * ratio), replace=False)
  test_irow = irow[idx]
  test_jcol = jcol[idx]

  r_copy = r.copy()

  for i, j in zip(test_irow, test_jcol):
      r_copy[i][j] = np.nan
  return r_copy, irow, jcol, test_irow, test_jcol

20% of the data is reserved for the test set.

In [8]:
r_train, irow, jcol, test_irow, test_jcol = train_test_split(r, 0.2)

100000 entries available

## Regression Model

When model training starts, bias values are generated randomly. After that, the model is created by updating it in the fitLinearRegression method.

In [9]:
userBias, itemBias = np.random.rand(r.shape[0]), np.random.rand(r.shape[1])
userBias, itemBias = fitLinearRegression(r_train, userBias, itemBias, irow, jcol, learning_rate = 0.001)

 10%|█         | 20/200 [00:19<03:22,  1.13s/it]

iteration 20, Total Loss: 41223.92939779688


 20%|██        | 40/200 [00:28<01:15,  2.12it/s]

iteration 40, Total Loss: 36440.70675837151


 30%|███       | 60/200 [00:38<01:06,  2.11it/s]

...

iteration 160, Total Loss: 33511.14614231133


 90%|█████████ | 180/200 [01:35<00:09,  2.12it/s]

iteration 180, Total Loss: 33447.74757881435


100%|██████████| 200/200 [01:44<00:00,  1.91it/s]

iteration 200, Total Loss: 33400.91967750449


##Test Set Prediction

Loss value and Mean Squared Error calculation are made on the test data that the model has not seen before.

In [10]:
def squareError(y, y_pred):
  return (y-y_pred)**2

In [11]:
seRegression, lossRegression = [], []
for i, j in zip(test_irow, test_jcol):
    y = r[i, j]

    y_pred = userBias[i] + itemBias[j]
    seRegression.append(squareError(y, y_pred))
    lossRegression.append(loss(r, userBias, itemBias, i, j))

In [12]:
cons.print(f"Regression Loss: {np.sum(np.array(lossRegression)):.5f}")
cons.print(f"Squared Error Regression: {np.sum(np.array(seRegression)):.5f}")

Regression Loss: 8960.62906

Squared Error Regression: 17921.25812

#Regularization Model

If we add a Regularization Term to the Loss Function and write a new Loss Function to avoid the risk of overfit:

## Loss Function

$$Loss Function: {\dfrac{1}{2}} \sum_{i,j \in S }  (r_{i,j} - \hat{r}_{i,j})^2 + {\dfrac{\lambda}{2}(\sum_{i=1}^m (\beta^{user}_i)^2 + \sum_{j=1}^n (\beta^{item}_j)^2) }$$

In [13]:
def lossRegularization(r: np.ndarray, userbias: np.ndarray, itembias: np.ndarray, i: int, j: int, lam: float):
  return ((r[i, j] - userbias[i] - itembias[j]) ** 2) / 2 + (lam/2) * (userbias[i] ** 2 + itembias[j]**2)

## Gradient

If we take the derivative of the Loss Function with respect to $\beta^{user}$ to calculate the gradient: <br> <br>

$$ \frac{\partial L}{\partial \beta^{user}} = -\sum_{i,j \in S }  (r_{i,j} - \beta^{user}_i - \beta^{item}_j) + \lambda * \sum_{i=1}^m  \beta^{user}_i$$

When we derivative the Loss function with respect to $\beta^{item}$ to calculate the gradient: <br><br>

$$\frac{\partial L}{\partial \beta^{item}} =  -\sum_{i,j \in S }  (r_{i,j} - \beta^{user}_i - \beta^{item}_j) + \lambda * \sum_{j=1}^n  \beta^{item}_j$$

In [14]:
def gradientRegularization(r: np.ndarray, userbias: np.ndarray, itembias: np.ndarray, i: int, j: int, lam: float, bias):
  return -(r[i, j] - userbias[i] - itembias[j]) + lam * bias

In [15]:
def fitRegularizationModel(r: np.ndarray, userbias: np.ndarray, itembias: np.ndarray, irow: np.ndarray, jcol: np.ndarray, learning_rate = 0.001, lam: float = 0.1, epoch: int = 200) -> np.ndarray:
    for t in trange(epoch):
        userbias_prev = userbias.copy()
        itembias_prev = itembias.copy()
        lossArr = []
        for i, j in zip(irow, jcol):
          if np.isnan(r[i, j]):
            continue
          lossArr.append(lossRegularization(r, userbias, itembias, i, j, lam))
          # Gradient for user
          gradientUser = gradientRegularization(r, userbias, itembias, i, j, lam, userbias[i])
          # Gradient for item
          gradientItem = gradientRegularization(r, userbias, itembias, i, j, lam, itembias[j])
          # Stochastic Gradient Descent
          userbias[i] = userbias[i] - learning_rate * gradientUser
          itembias[j] = itembias[j] - learning_rate * gradientItem
          
        if (t+1) % 20 == 0:
          print(f"iteration {t+1}, Total Loss: {np.sum(lossArr)}")
        if np.linalg.norm(userbias - userbias_prev) < 0.001 or np.linalg.norm(itembias - itembias_prev) < 0.001:
            print(f"I do early stopping at iteration {t+1}")
            break
            
    return userbias, itembias

## Train-Validation Split

When doing lambda optimization, the train set is divided into train set and validation set. Trained models are tested on validation set.

In [16]:
r_trainReg, irowReg, jcolReg, val_irow, val_jcol = train_test_split(r_train, 0.1)

80000 entries available

In [17]:
print(f"Number of entries available in Train Set {len(np.where(~np.isnan(r_trainReg))[0])}")

Number of entries available in Train Set 72000


In [18]:
print(f"Number of entries available in Validation Set {len(val_irow)}")

Number of entries available in Validation Set 8000


## Lambda Optimization

Models are created with different lambda values. The performance of the models is tested on the validation set.

In [19]:
lamArr = np.linspace(0.1, 0.5, 9)

In [20]:
lossSumLam = []

for lam in lamArr:
  lossLam = []
  print("\nLambda: ", str(lam), "\n")
  userBiasReg, itemBiasReg = np.random.rand(r.shape[0]), np.random.rand(r.shape[1])
  userBiasReg, itemBiasReg = fitRegularizationModel(r_trainReg, userBiasReg, itemBiasReg, irowReg, jcolReg, learning_rate = 0.001, lam = lam)
  for i, j in zip(val_irow, val_jcol):
    lossLam.append(lossRegularization(r, userBiasReg, itemBiasReg, i, j, lam))
    #lossLam.append((((r[i, j] - userBiasReg[i] - itemBiasReg[j]) ** 2) / 2))
  lossSumLam.append(np.sum(np.array(lossLam)))


Lambda:  0.1 



 10%|█         | 20/200 [00:11<01:55,  1.56it/s]

iteration 20, Total Loss: 59551.023788697756


 20%|██        | 40/200 [00:24<01:33,  1.71it/s]

iteration 40, Total Loss: 55357.959202027014


...

iteration 160, Total Loss: 124793.92016583387


 90%|█████████ | 180/200 [01:45<00:11,  1.71it/s]

iteration 180, Total Loss: 124769.01333955748


100%|██████████| 200/200 [01:56<00:00,  1.71it/s]

iteration 200, Total Loss: 124749.8989546836


Loss Values obtained on the performance validation set of the models

In [21]:
lossSumLam

[6045.054248677402,
 7213.971151058487,
 8323.022853815964,
 9381.672619250588,
 10392.168820094794,
 11359.770812520886,
 12286.95144942747,
 13170.059167386002,
 14021.812989160091]

The Lambda value that provides the Minimum Loss is selected.

In [22]:
optimalLam = lamArr[np.argsort(lossSumLam)[0]]

## Final Regularized Model

Train and Validation sets are trained together with the selected Lambda.

In [23]:
userBiasReg, itemBiasReg = np.random.rand(r.shape[0]), np.random.rand(r.shape[1])
userBiasReg, itemBiasReg = fitRegularizationModel(r_train, userBiasReg, itemBiasReg, irow, jcol, lam = optimalLam)

 10%|█         | 20/200 [00:13<02:02,  1.47it/s]

iteration 20, Total Loss: 65491.96670319197


 20%|██        | 40/200 [00:27<01:47,  1.48it/s]

iteration 40, Total Loss: 61224.43611159066


 30%|███       | 60/200 [00:40<01:34,  1.49it/s]

...

iteration 160, Total Loss: 58785.499754122815


 90%|█████████ | 180/200 [02:01<00:13,  1.46it/s]

iteration 180, Total Loss: 58736.93673406892


100%|██████████| 200/200 [02:15<00:00,  1.48it/s]

iteration 200, Total Loss: 58700.84897185277


##Test Set Prediction

Loss value and Mean Squared Error calculation are made on the test data that the model has not seen before.

In [24]:
seRegularizedModel, lossRegularizedModel = [], []
for i, j in zip(irow, jcol):
    y = r[i, j]

    y_pred = userBiasReg[i] + itemBiasReg[j]
    seRegularizedModel.append(squareError(y, y_pred))
    lossRegularizedModel.append(lossRegularization(r, userBiasReg, itemBiasReg, i, j, optimalLam))

In [25]:
cons.print(f"Loss of Regularized Regression : {np.sum(np.array(lossRegularizedModel)):.5f}")
cons.print(f"Squared Error of Regularized Regression: {np.sum(np.array(seRegularizedModel)):.5f}")

Loss of Regularized Regression : 73882.88167

Squared Error of Regularized Regression: 87849.84167